Ce script permet de parser les fiches métiers au format XML envoyé par l'Onisep.
Le résultat donne des fiches au format JSON copiés dans le dossier 'Parsing output'.

In [1]:
import os
import logging
from lxml import etree
import json

In [2]:
# Préparation des chemins utiles
#/PycharmProjects/ImpalaPoleEmploi2/Impala/Onisep - tSNE et clustering /MetiersOnisep
dir_path = os.getcwd()
# Dossier métiers Onisep
metiers_dir_path = os.path.join(dir_path, '0 - data/MetiersOnisep')
# Dossier output
output_dir_path = os.path.join(dir_path, '00 - Parsing output')

# Une fiche pour tester
fiche = os.path.join(metiers_dir_path,'10002.xml')
# Liste des fiches métiers
list_xml_files = os.listdir(metiers_dir_path)

In [4]:
log = logging.getLogger()
map(log.removeHandler, log.handlers[:])
map(log.removeFilter, log.filters[:])

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    filemode='w')

# define a Handler which writes INFO messages or higher to the sys.stderr
console = logging.StreamHandler()
console.setLevel(logging.INFO)
# set a format which is simpler for console use
formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
# tell the handler to use this format
console.setFormatter(formatter)
# add the handler to the root logger
log.addHandler(console)

In [6]:
#This function put all the information of the xml file into a dictionnary
def parse_xml_file(fiche):
    fiche_dict = {}
    #log.info(fiche)
    #tree = etree.parse(metiers_dir_path + '/' + fiche)
    tree = etree.parse(fiche)
    root = tree.getroot()
    #log.info(root)Statuts
    
    fiche_dict["Métier"] = root.find('MET_LIBELLE_METIER').text
    
    if root.find('MET_CENTRE_INTERET').text is not None:
        fiche_dict["Centres d'intérêt"] = root.find('MET_CENTRE_INTERET').text.split('/')
    if root.find('MET_CATEGORIE_SOCIO_PRO').text is not None:
        fiche_dict["Statuts"] = root.find('MET_CATEGORIE_SOCIO_PRO').text.split('/')
    if root.find('MET_DOMAINE').text is not None:
        fiche_dict["Fonctions"] = root.find('MET_DOMAINE').text.split('/')
    fiche_dict["Secteurs d'activité"] = root.find('MET_SECTEUR_ACTIVITE').text.split('/') 
    if root.find('MET_DISCIPLINE').text is not None:
        fiche_dict["Disciplines"] = root.find('MET_DISCIPLINE').text.split('/')
    fiche_dict["Groupe Formation Emploi"] = root.find('MET_GFE').text.split('/')
    fiche_dict["lien_onisep"] = root.find('lien_onisep').text
   
    fiche_dict["rome"] = {}    
    bloc_rome = root.find('ROME')
    for record in bloc_rome.iter('record'):
        code_rome = record.find('FAMROM_CODE_ROME').text
        libelle = record.find('FAMROM_LIBELLE').text
        fiche_dict["rome"][code_rome] = libelle

    fiche_dict["Métiers associés"] = []
    for metier in root.iter('METCONN_LIBELLE_METIER'):
        fiche_dict["Métiers associés"].append(metier.text)
    
    fiche_dict["Descriptif"] = {}
    for bloc in root.iter('MET_NATURE_DESCRIPTIF'):
        name = "Intro"
        for x in bloc:
            if x.tag=='inter':
                name = x.text
                fiche_dict["Descriptif"][name] = []
            else:
                fiche_dict["Descriptif"][name] = []
                fiche_dict["Descriptif"][name].append(x.text)
                
    fiche_dict["Accès au métier"] = {}
    for bloc in root.iter('MET_ACCES_DESCRIPTIF'):
        for x in bloc:
            if x.find('accr_p') is not None:
                fiche_dict["Accès au métier"] = x.find('accr_p').text
        
    fiche_dict["Vie professionnelle"] = {}
    for bloc in root.iter('MET_VIE_PRO_DESCRIPTIF'):
        for x in bloc:
            if x.tag=='inter':
                name = x.text
                fiche_dict["Vie professionnelle"][name] = []
            elif x.text is not '\n':
                fiche_dict["Vie professionnelle"][name].append(x.text)
                
    return fiche_dict

parse_xml_file(fiche)

{'Accès au métier': "Si plusieurs diplômes préparent à ce métier, il faut savoir qu'un niveau d'études plus élevé conduit à des postes avec davantage de responsabilités.",
 "Centres d'intérêt": ['fabriquer, construire',
  'travailler un matériau',
  'exercer un métier artistique ou créatif',
  'concevoir, utiliser des technologies industrielles de pointe',
  'faire du dessin'],
 'Descriptif': {'Concevoir les produits': ["Au sein d'un bureau d'études, il contribue à la conception et au design des produits. Il réalise des plans à partir de modèles existants, en fournissant toutes les indications nécessaires à la fabrication d'un meuble, d'une porte, d'une fenêtre... Cela concerne les études avec plan grandeur nature, le prototype, les méthodes, le contrôle qualité, la gestion des stocks... "],
  'Différentes missions': ["Définir des méthodes pour concevoir et fabriquer différents ouvrages en bois, participer au lancement et au suivi de la fabrication : tels sont les objectifs du technici

In [7]:
# Boucle pour parser toutes les fiches
for item in list_xml_files:
    metier_path = os.path.join(metiers_dir_path, item)
    metier = parse_xml_file(metier_path)
    metier_output_path = os.path.join(output_dir_path, item) + '.json'
    #print(metier_output_path)
    with open(metier_output_path, 'w') as f:
        json.dump(metier, f, indent=4)